# Data Cleansing

In [1]:
# importing packages
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Loading data
file_path = Path('./Data/data_2019_original.csv')
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,21562,1,1,2019,1,1,13:05,$51.94,Online Transaction,4241336128694185533,ONLINE,NaN,NaN,4814,NaN,No
1,21563,1,1,2019,1,3,13:57,$18.85,Chip Transaction,-6738340320657348028,Corona,NY,11368.0,7538,NaN,No
2,21564,1,1,2019,1,4,18:43,$62.36,Chip Transaction,-5023497618971072366,Little Neck,NY,11363.0,5812,NaN,No
3,21565,1,1,2019,1,5,15:33,$230.07,Chip Transaction,4937362508976226544,Little Neck,NY,11363.0,5300,NaN,No
4,21566,1,1,2019,1,5,18:57,$63.75,Chip Transaction,-7052069146128772826,Little Neck,NY,11363.0,5814,NaN,No


In [3]:
# Checking the data shape (rows x columns)
df.shape

(272118, 16)

In [4]:
# rename columns
df.rename(columns={
    'Is Fraud?':'isFraud',
    'Errors?':'Errors'
}, inplace=True)

In [5]:
# Dropping unwanted columns
df.drop(columns=['Unnamed: 0', 'Errors'], inplace=True)

In [6]:
# Dropping any duplicates
df.drop_duplicates(inplace=True)

In [7]:
# Checking data types
df.dtypes

User                int64
Card                int64
Year                int64
Month               int64
Day                 int64
Time               object
Amount             object
Use Chip           object
Merchant Name       int64
Merchant City      object
Merchant State     object
Zip               float64
MCC                 int64
isFraud            object
dtype: object

In [8]:
# Checking the Target variable class ratio
df['isFraud'].value_counts()

No     270031
Yes      2087
Name: isFraud, dtype: int64

This conclude that this is a highly imbalanced dataset with around 1:128 Imbalanced Ratio

In [9]:
# Cheeck Null/NaN values
df.isna().sum()

User                  0
Card                  0
Year                  0
Month                 0
Day                   0
Time                  0
Amount                0
Use Chip              0
Merchant Name         0
Merchant City         0
Merchant State    28625
Zip               32334
MCC                   0
isFraud               0
dtype: int64

In [10]:
# Further inspecting Merchant State based on Merchant City
df.loc[df['Merchant State'].isna(), 'Merchant City'].value_counts()

ONLINE    28625
Name: Merchant City, dtype: int64

It is clear that Merchant State data is missing because of ONLINE Transactions

In [11]:
# Fill the missing data with 0
df['Merchant State'].fillna('UNKNOWN', inplace=True)

In [12]:
# Dropping Zip code since Merchant State and City information available
df.drop(columns=['Zip'], inplace=True)

In [13]:
# Split the Time column into Hour and Minutes
df['Hour'] = df['Time'].apply(lambda x: x.split(':')[0])
df['Minutes'] = df['Time'].apply(lambda x: x.split(':')[1])

In [14]:
# Remove Time columns
df.drop(columns=['Time'], inplace=True)

In [15]:
# Re-arrange the columns
all_cols = list(df.columns)
all_cols.insert(5,'Hour')
all_cols.insert(6,'Minutes')
del all_cols[-2:]

df = df[all_cols]

In [16]:
# Format the Amount column and chane the data type to float
df['Amount'] = df['Amount'].str[1:].astype(np.float)

In [17]:
# Merchant Name column is a categorical variable but in numbers so label them for readability
merchant_name_encoder = LabelEncoder()
df['Merchant Name'] = merchant_name_encoder.fit_transform(df['Merchant Name'])

In [18]:
# Checking a sample
df.head()

,User,Card,Year,Month,Day,Hour,Minutes,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,MCC,isFraud
0,1,1,2019,1,1,13,05,51.94,Online Transaction,5531,ONLINE,UNKNOWN,4814,No
1,1,1,2019,1,3,13,57,18.85,Chip Transaction,1023,Corona,NY,7538,No
2,1,1,2019,1,4,18,43,62.36,Chip Transaction,1710,Little Neck,NY,5812,No
3,1,1,2019,1,5,15,33,230.07,Chip Transaction,5840,Little Neck,NY,5300,No
4,1,1,2019,1,5,18,57,63.75,Chip Transaction,895,Little Neck,NY,5814,No


In [19]:
# Saving the cleaned data as a CSV file
df.to_csv('./Data/data_2019.csv', index=False)